<h1> Bike share data analysis sample case </h1>


<h3>Author: Luis Gonzalez </h3>
<p></p>
Email: luisf.gonzalezv@yahoo.com
<p></p>
Date: 18/12/2023

This notebook contains the step-by-step process of a data analysis sample case made on free access divvy data from the year 2022, a bike share company from Chicago. 
<p></p>
the original dataset is avaiblable at: https://divvy-tripdata.s3.amazonaws.com/index.html .
The purpose of this data analysis, as stated in the main report, is to identify the main differences between casual customers and members, as well as finding possible insights on how to market membserhips for casual bike share customers.  

<h3>Step 1: Data Collection </h3>
<p></p>
From the webpage shown above, we can see the data is downloadable by month, as well as by quarter, and on the moment this report was made, the first available dataset was for april 2020 and the last dataset was from november 2023. We choose to analyze 2022, as the last complete year available. 
<p></p>
The data is available as a downloadable zip folder. For convenience, it´s best to simply download these zip folders, and gather the .csv files from every month. We also name each file with the same format, as it will be useful for the data handling process later on. 
<p></p>
We name the file "2022mo-divvy-tripdata.csv", where mo is a number from 01 to 12 (the 0 is important) for each month of the year.

<h3> Step 2: Data Understanding </h3>
<p></p>
 Before wrangling the data, it´s important to understand its contents and properties. For exploratory analysis of the files, we can use the pandas library. We now create a list with all the file names in the mentioned format, which makes it easier for looping. 

In [1]:
i=1
files=list()
while i<13:
    if i<10:
        fname="20220"+str(i)+"-divvy-tripdata.csv"
    else :
        fname="2022"+str(i)+"-divvy-tripdata.csv"
    files.append(fname)
    i=i+1

In [2]:
len (files)

12

In [3]:
# import required library
import pandas as pd

In [4]:
# let´s check the columns for this dataset
df = pd.read_csv(files[0], header=0)
df.columns

Index(['ride_id', 'rideable_type', 'started_at', 'ended_at',
       'start_station_name', 'start_station_id', 'end_station_name',
       'end_station_id', 'start_lat', 'start_lng', 'end_lat', 'end_lng',
       'member_casual'],
      dtype='object')

In [5]:
df.head()

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual
0,C2F7DD78E82EC875,electric_bike,2022-01-13 11:59:47,2022-01-13 12:02:44,Glenwood Ave & Touhy Ave,525,Clark St & Touhy Ave,RP-007,42.012800,-87.665906,42.012560,-87.674367,casual
1,A6CF8980A652D272,electric_bike,2022-01-10 08:41:56,2022-01-10 08:46:17,Glenwood Ave & Touhy Ave,525,Clark St & Touhy Ave,RP-007,42.012763,-87.665967,42.012560,-87.674367,casual
2,BD0F91DFF741C66D,classic_bike,2022-01-25 04:53:40,2022-01-25 04:58:01,Sheffield Ave & Fullerton Ave,TA1306000016,Greenview Ave & Fullerton Ave,TA1307000001,41.925602,-87.653708,41.925330,-87.665800,member
3,CBB80ED419105406,classic_bike,2022-01-04 00:18:04,2022-01-04 00:33:00,Clark St & Bryn Mawr Ave,KA1504000151,Paulina St & Montrose Ave,TA1309000021,41.983593,-87.669154,41.961507,-87.671387,casual
4,DDC963BFDDA51EEA,classic_bike,2022-01-20 01:31:10,2022-01-20 01:37:12,Michigan Ave & Jackson Blvd,TA1309000002,State St & Randolph St,TA1305000029,41.877850,-87.624080,41.884621,-87.627834,member


In [6]:
df.shape

(103770, 13)

In [7]:
df["rideable_type"].value_counts()

classic_bike     55067
electric_bike    47742
docked_bike        961
Name: rideable_type, dtype: int64

From exploratory analysis of this dataset we conclude it contains 13 columns or fiels, with over 100 thousand observations for each month. To avoid making the code unnecessarily slow, we don´t open the other months, but they all contain the same columns, which is the most important thing. The columns contain the following information:
- ride_id: A unique string for every distinct trip or observation. It can be used as a primary key if moved to a database.
- rideable_type: the type of bike used, either classic, electric or docked.
- started_at and ended_at: a datetime format value for when the trip started or ended.
- start_station_name and end_station_name: String values for the location of each station where the trips took place.
- start_station_id	and end_station_id: Unique string values for each station name.
- start_lat, start_lng, end_lat, end_lng: The spatial coordinates for each trip registered. "Lat" stands for latitude, and "lng" for longitude.
- member_casual: Identifies each user as a member (i.e. with a subscription to the service) or a casual user.



In order to know how many observations we really have, we can run the following cell of code:

In [8]:
count=0
for i in files:
    df = pd.read_csv(i, header=0)
    count=count+df.shape[0]
print("In total, there are "+str(count)+" observations for the annual dataset.")

In total, there are 5667717 observations for the annual dataset.


We discover, the size of the combined dataset is quite extense, over 5 million obervations, each with 13 attributes. We want to use a data analysis method that is able to move through the dataset and make changes as fast as possible. 

<h3>Step 3: Data Preparation</h3>
<p></p>
With the size of the dataset in mind, it´s a good idea to use an SQL database in conjucntion with python, for quick and effective handling of medium to large datasets. Python has a built-in library for this, using free-access sqlite3 software.


Now we import the relevant modules for the entire process.

In [9]:
import sqlite3
import csv

Connect to an sqlite3 database, and create a cursor object. We then create empty table schemas, with the appropriate column formatting for efficient memory storage within the database. Notice that a separate table, stations, is created for storing the string values of each distinct station. This will help create quicker CRUD operations on the main table, total, as strings take up more space in general than other SQL data formats.   

In [10]:
conn = sqlite3.connect('divvy_public.sqlite') 
cur = conn.cursor()
conn = sqlite3.connect('divvy_public.sqlite') 
cur = conn.cursor() #file handle for the DB cursor object

In [11]:
#  create table schemas
cur.executescript('''DROP TABLE IF EXISTS stations;
                  CREATE TABLE stations (
                      station_id TINYTEXT NOT NULL PRIMARY KEY UNIQUE,
                      station_name TINYTEXT,
                      latitude REAL,
                      longitude REAL, 
                      times_visited INTEGER DEFAULT 0);
                  ''')
                  
cur.executescript('''DROP TABLE IF EXISTS total;
                    CREATE TABLE total (
                        ride_id TINYTEXT NOT NULL PRIMARY KEY UNIQUE,
                        rideable_type TINYTEXT,
                        started_at DATETIME,
                        ended_at DATETIME,
                        start_station_id TINYTEXT,
                        end_station_id TINYTEXT,
                        start_lat REAL,
                        start_lng REAL,
                        end_lat REAL,
                        end_lng REAL,
                        membership TINYTEXT,
                        trip_duration INTEGER DEFAULT 0,
                        trip_distance INTEGER DEFAULT 0
                        );''')
    
print("Database and tables created successfully.")

Database and tables created successfully.


Loop through each file in the files list. Then run a nested loop, where you loop through every line in each file, except for the first line (the headers). For each valid line, insert the record into our existing database. This cell of code can take several minutes to complete. 

In [12]:
for i in files:
    with open(i, mode="r") as data:
        read = csv.reader(data, delimiter=',')
        count = 0
# row is a list of the actual data, ordered like follows:
# ['ride_id': 0 , 'rideable_type': 1, 'started_at': 2, 'ended_at':3, 
#'start_station_name':4, 'start_station_id':5, 'end_station_name':6, 
#'end_station_id':7, 'start_lat':8, 'start_lng':9, 'end_lat':10, 
#'end_lng':11,#'member_casual': 12]
        for row in read: 
# count must be greater than 0, so we don´t insert the headers
            if count==0: 
                count=count+1
                #continue
# fill table stations, with start_station
# the "or ignore" allows for only new records to be added
            cur.execute('''INSERT OR IGNORE INTO 
                              stations (station_id, station_name, 
                                    latitude, longitude)
                              VALUES (?, ?, ?, ?)
                              ''',(row[5], row[4], row[8], row[9]))
# end_station
            cur.execute('''INSERT OR IGNORE INTO 
                              stations (station_id, station_name, 
                                    latitude, longitude)
                              VALUES (?, ?, ?, ?)
                              ''',(row[7], row[6], row[10], row[11]))
# fill table total with all the months
            cur.execute('''INSERT OR IGNORE INTO 
total (ride_id, rideable_type, started_at, ended_at, start_station_id, 
       end_station_id, start_lat, start_lng, end_lat, end_lng, membership)
                              VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ? )
        ''',(row[0], row[1], row[2], row[3],row[5],row[7], row[8], row[9], row[10], row[11], row[12]))
            count=count+1
#this command saves changes to the db. If you commit through every row,
# the whole code might take hours to run
        conn.commit()
        print ("Success. "+str(count)+" rows were processed for file "+str(i))
conn.close()
print("All records from the csv files have been inserted into the database correctly. ")

Success. 103772 rows were processed for file 202201-divvy-tripdata.csv
Success. 115611 rows were processed for file 202202-divvy-tripdata.csv
Success. 284044 rows were processed for file 202203-divvy-tripdata.csv
Success. 371251 rows were processed for file 202204-divvy-tripdata.csv
Success. 634860 rows were processed for file 202205-divvy-tripdata.csv
Success. 769206 rows were processed for file 202206-divvy-tripdata.csv
Success. 823490 rows were processed for file 202207-divvy-tripdata.csv
Success. 785934 rows were processed for file 202208-divvy-tripdata.csv
Success. 701341 rows were processed for file 202209-divvy-tripdata.csv
Success. 558687 rows were processed for file 202210-divvy-tripdata.csv
Success. 337737 rows were processed for file 202211-divvy-tripdata.csv
Success. 181808 rows were processed for file 202212-divvy-tripdata.csv
All records from the csv files have been inserted into the database correctly. 


Now we have every record from the dataset available on the same database, we can more easily analyze the data, as well as screen for incorrect observations. For this second part of exploratory data analysis, it´s very useful to use SQL magic. An added advantage of this method is, when updating the database every command is committed automatically, which isn´t the case using the standard sqlite3 library.

In [13]:
# if you don´t have the library installed:
#!pip3 install ipython-sql

In [14]:
%load_ext sql
%sql sqlite:///divvy_public.sqlite

In [15]:
%%sql
SELECT
    count(*)
FROM
    total;

 * sqlite:///divvy_public.sqlite
Done.


count(*)
5667718


Initially, there are over 5 million rows in the database. Seeing how we have start and end times for each ride, we can try to calculate the duration of each trip. The following cells of code do that, rounded to the nearest minute. 

In [16]:
%%sql 
UPDATE
    total
SET
    trip_duration=(JULIANDAY(ended_at) - JULIANDAY(started_at))*1440;


 * sqlite:///divvy_public.sqlite
5667718 rows affected.


[]

In [17]:
%%sql
UPDATE
    total
SET
    trip_duration=ROUND(trip_duration,0)

 * sqlite:///divvy_public.sqlite
5667718 rows affected.


[]

Using our newly created column, we can start cleaning the database for incorrect entries. For starters, it doesn´t make sense that a record should have a negative trip duration value, or a value that´s too small. For example, it´s more likely a trip duration of one minute was due to a racking or unracking error form a user, than an actual trip.

In [18]:
%%sql 
DELETE FROM
    total
WHERE
    trip_duration<=1;

 * sqlite:///divvy_public.sqlite
161987 rows affected.


[]

Following the same train of thought, what about very short trips that start and end at the same station? We can use an educated guess, and establish that trips of less than 5 minutes and the same start and end locations are also not valid.

In [19]:
%%sql 
DELETE FROM
    total
WHERE
    trip_duration<=5
AND
    start_station_id=end_station_id;

 * sqlite:///divvy_public.sqlite
146625 rows affected.


[]

Now, we consider the fields of latitude and longitude. As the dataset is from the city of Chicago, which is located at approximately [41.8, -87.6], registers that contain any 0 in their coordinates are also invalid.

In [20]:
%%sql
DELETE FROM
  total
WHERE
  start_lat=0 OR end_lat=0 OR start_lng=0 OR end_lng=0;

 * sqlite:///divvy_public.sqlite
6 rows affected.


[]

Now, we can try to calculate the actual distance of each trip. The earth closely resembles a sphere, which is non-euclidean geometry. Therefore, to calculate the shortest distance between two points we use the <i>geodesic</i>. 
The webpage https://www.geeksforgeeks.org/program-distance-two-points-earth/ offers a great explanation, as well as a code implementation for python of how to calculate a geodesic given start and end coordinates. For this part, we can´t use sql magic, as we need to make the geodesic calculations outside sqlite, in python. This code takes several minutes to complete.

In [21]:
#  this cell of code gives us valuable status checks on code that takes some time to run
checkpoints=[0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
check_status=dict()
for i in checkpoints:
    check_status[i]=False

#if we don´t change the dict status to True, it will print many times 
# instead of just one
def checkpoint_notifications(row_count, total_count):
    progress=round((row_count/total_count),1)
    if progress in check_status and check_status[progress]==False:
        value=(round(progress,1))*100
        print (str(value)+"% completed.")
        check_status[progress]=True
        conn.commit()

In [22]:
from math import radians, cos, sin, asin, sqrt
def distance(lat1, lat2, lon1, lon2):
     
    # The math module contains a function named
    # radians which converts from degrees to radians.
    lon1 = radians(lon1)
    lon2 = radians(lon2)
    lat1 = radians(lat1)
    lat2 = radians(lat2)
      
    # Haversine formula
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
 
    c = 2 * asin(sqrt(a))
    
    # Radius of earth in kilometers. Use 3956 for miles
    r = 6371
      
    # calculate the distance in km
    return(c * r)


#start the connection to the database
conn = sqlite3.connect('divvy_public.sqlite') 
cur = conn.cursor()
# this statement saves the total rows to the variable total_count
cur.execute('''SELECT COUNT(*) FROM total;''')
total_count=cur.fetchone()[0]

# with this statement, I can obtain the inputs for the distance function
cur.execute('''SELECT start_lat, end_lat, start_lng, end_lng, ride_id
            FROM total;
                      ''')
# cur.fetchall is a list with the elements I asked, for every row
table=cur.fetchall()
row_count=0
for row in table:
# make the calculation. If it´s unsuccesful for some reason, return a 0
# for distance. We can clean those values later directly in the db
    try:
        lat1= float(row[0])
        lat2=float(row[1])
        lon1=float(row[2])
        lon2=float(row[3])
        dist=round(distance(lat1,lat2,lon1,lon2),2)
    except:
        dist=0
    cur.execute('''UPDATE total SET trip_distance 
                =? WHERE ride_id=?''', (dist,row[4]))
    row_count=row_count+1
# call the function we defiend for progress notifications
    checkpoint_notifications(row_count, total_count)

conn.commit()
print("Success. "+str(total_count)+ " rows were affected in the database.")
conn.close()

10.0% completed.
20.0% completed.
30.0% completed.
40.0% completed.
50.0% completed.
60.0% completed.
70.0% completed.
80.0% completed.
90.0% completed.
Success. 5359100 rows were affected in the database.


With this new field, we can introduce additional data cleaning queries.

In [23]:
%%sql
DELETE FROM
    total
WHERE
    trip_distance=0 AND 
        start_station_id !=end_station_id;

 * sqlite:///divvy_public.sqlite
13083 rows affected.


[]

We now wish to work on the stations table. It would be useful for future analysis to know how many times each station was visited. Knowing the most visited stations can also aid a marketing team in deciding where it is best to place advertising for casual users to become members, and increase customer retention.

In [24]:
#start the connection to the database
conn = sqlite3.connect('divvy_public.sqlite') 
cur = conn.cursor()
# get the number of columns
cur.execute('''SELECT COUNT(*) FROM total;''')
total_count=cur.fetchone()[0]

# reset the count, in case there´s already values
cur.execute('''UPDATE stations
                SET times_visited=0;''')


# run a query to get all start_station ids from total
cur.execute('''SELECT start_station_id, end_station_id
                FROM total;''')
data1=cur.fetchall()
row_count=0
for row in data1:
    id1=str(row[0])
    id2=str(row[1])    
#update count for every ocurrence of start_station      
    cur.execute('''UPDATE stations
                    SET times_visited=times_visited+1
                    WHERE station_id= ?;''', [id1])
# the same, but also for the end_station
    cur.execute('''UPDATE stations
                     SET times_visited=times_visited+1
                     WHERE station_id= ?;''', [id2])
    row_count=row_count+1
# call the function we defiend for progress notifications
    checkpoint_notifications(row_count, total_count)
conn.close()


We run additional data cleaning, taking advantage of this newly-created field. 

In [25]:
%%sql
DELETE FROM
    stations
WHERE
    times_visited=0;

 * sqlite:///divvy_public.sqlite
1325 rows affected.


[]

<h3>Step 4: Finding insights (modeling)</h3>
<p></p>
Now the data is cleaned, we can look to derive insights from it, in accordance with the business goals of this sample case. We can start by identifying the 10 most popular trips from registered stations. 

In [ ]:
%%sql
DROP TABLE IF EXISTS 
    top_trips;
CREATE TABLE 
    top_trips AS
SELECT
    start_station_id, end_station_id, COUNT(*) AS trip_count
FROM
    total
WHERE
    LENGTH(start_station_id)>1 AND LENGTH(end_station_id)>1
GROUP BY
    start_station_id, end_station_id
ORDER BY 
    trip_count DESC
LIMIT 10;

 * sqlite:///divvy_public.sqlite
Done.


We now match the station id to its actual name.

In [ ]:
%%sql
ALTER TABLE 
    top_trips
ADD 
    start_station_name TINYTEXT;

In [ ]:
%%sql
ALTER TABLE 
    top_trips
ADD 
    end_station_name TINYTEXT;

For matching the name to the id, we can use placeholders in python.

In [ ]:
conn = sqlite3.connect('divvy_public.sqlite') 
cur = conn.cursor()
cur.execute('''SELECT start_station_id,
                end_station_id 
                FROM top_trips;''')
data=cur.fetchall()
for row in data:
    start_id=row[0]
    end_id=row[1]
#  find the station name corresponding to the id in table stations,
#  then insert it into top_trips
    cur.execute('''SELECT station_name
                    FROM stations
                    WHERE station_id= ?;''',[start_id])
    start_name=cur.fetchone()[0]
    cur.execute('''UPDATE top_trips
                    SET start_station_name= ?
                    WHERE start_station_id=?
                ;''',(start_name,start_id))
#  the same, but for end_station
    cur.execute('''SELECT station_name
                    FROM stations
                    WHERE station_id= ?;''',[end_id])
    end_name=cur.fetchone()[0]
    cur.execute('''UPDATE top_trips
                    SET end_station_name= ?
                    WHERE end_station_id=?
                ;''',(end_name,end_id))
    conn.commit()
conn.close()

We now check for a preview of this newly-created table:

In [ ]:
%%sql
SELECT * FROM 
    top_trips;

We can also check the stations table, to gather the most visited places:

In [ ]:
%%sql
SELECT*FROM
    stations
WHERE
    LENGTH(station_name)>1
ORDER BY 
    times_visited DESC
LIMIT 10;


We now know the Streeter Dr & Grand Ave station is by far the most popular divvy bike share station, and possibly where future advertising efforts directed to casual members should be concentrated. But what about casual users and paid members? What differentiates them from each other?

In [ ]:
%%sql 
SELECT* 
FROM 
    total
LIMIT 5;

In [ ]:
%%sql
SELECT
    membership, COUNT(*) AS "total trips", AVG(trip_duration), AVG(trip_distance)
FROM 
    total
GROUP BY
    membership;

We can observe that members take more trips in total than casual users. They also take similar distance trips, but in much less time. A possible explanation for this, is that members need more frequent use of the bikes for their day-to-day endeavours, therefore they take more trips than casual users. They also take less time, because their trips are more likely to be previously planned. Casual users, on the other hand, may use the bikes on a more "ad hoc" basis, involving less planning and less bike use, and take more time to complete their trips. 

In [ ]:
%%sql 
SELECT 
    rideable_type, membership, count(*) AS "total trips"
FROM 
    total
GROUP BY 
    rideable_type, membership

We can now observe casual users tend to use more electric bikes, whereas members tend to prefer classic bikes. Again, we can try to explain this by considering casual users might need the rental service for an unplanned event, therefore appreciate more electric bikes for getting to their destination faster. 
<p></p>
Now, we have gathered sufficient information on the differences between casuals and members, and can proceed to share our findings. These can be graphically illustrated using pandas dataframes and plotting libraries like seaborn or matplotlib, but on this occation we choose to employ Tableau Public, a very popular business analytics tool with a user-friendly interface, capable of producing stunning visualizations.

You can check the visualization results at https://public.tableau.com/app/profile/luis.gonzalez6272/viz/Divvypublicdata2022/top_stations, or continue on this folder for the complete results. 

<p>
Thank you for your interest in this project :)
    </p>